##  **`jamspell`** : librairie de correction d'orthographe contextuelle
* développée par Filipp Ozinov ([dépôt GitHub](https://github.com/bakwc/JamSpell))
* écrite en C++
* <u>prérequis</u> : installer les _bindings_ `swig` 
  * compilateur d'interface qui connecte des programmes écrits en C++ avec, en l'occurrence, le langage Python
* [instructions d'installation en local](https://github.com/bakwc/JamSpell/#usage)
  * pour contourner les problèmes d'installation potentiels du `swig` et, par conséquent, de `jamspell`, envisagez à utiliser Colab

### Installer `swig` 

In [1]:
!sudo apt-get install swig

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,455 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fr

### Installer `jamspell`

In [2]:
!sudo pip install jamspell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 25.9 MB/s 
  Created wheel for jamspell: filename=jamspell-0.0.12-cp37-cp37m-linux_x86_64.whl size=1347595 sha256=91e633385cfb78315a47b0b945ddd8a53e899fd50cd42ffeeca70a2a3b1f84a6
  Stored in directory: /root/.cache/pip/wheels/68/df/9c/9b335e69aa0f28e7f508ec0ebefadcc703f168beb52ae7ebe7
Successfully built jamspell


### Télécharger le modèle de langue français
* corpus d'actualités (300 000 phrases) + 300 000 phrases de wikipedia


In [3]:
!wget https://github.com/bakwc/JamSpell-models/raw/master/fr.tar.gz

--2022-05-27 14:08:14--  https://github.com/bakwc/JamSpell-models/raw/master/fr.tar.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/fr.tar.gz [following]
--2022-05-27 14:08:14--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/fr.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32421326 (31M) [application/octet-stream]
Saving to: ‘fr.tar.gz’

fr.tar.gz           100%[===================>]  30.92M  --.-KB/s    in 0.1s    

2022-05-27 14:08:14 (263 MB/s) - ‘fr.tar.gz’ saved [32421326/32421326]



* <u>recommandation</u> : entraîner son propre modèle (selon les [instructions dédiées](https://github.com/bakwc/JamSpell/#train)) au moins sur quelques millions de phrases pour obtenir une meilleure qualité

### Décompresser le fichier `tar` :

In [4]:
!tar -xvf fr.tar.gz

fr.bin


### Faire une première démo :


In [5]:
import jamspell 

jsp = jamspell.TSpellCorrector()   # initialiser le correcteur
assert jsp.LoadLangModel('fr.bin') # spécifier le modèle de langue

jsp.FixFragment("Bnojour, cecci est un tutorie")

'Bonjour, ceci est un tutoriel'

### Créer trois dossiers où : 
* `sample_in` : données d'entrée (texte à corriger)
* `sample_out` : données de sortie (texte corrigé)
* `csv` : tableur .csv avec les mots orthographiquement incorrects, leurs corrections et les fréquences d'apparence des erreurs


Ensuite, importer les fichiers à corriger et les mettre dans `sample_in`.

In [6]:
import os

newpath = r'/content/drive/MyDrive/sample_in' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

newpath2 = r'/content/drive/MyDrive/sample_out' 
if not os.path.exists(newpath2):
    os.makedirs(newpath2)

newpath3 = r'/content/drive/MyDrive/csv' 
if not os.path.exists(newpath3):
    os.makedirs(newpath3)

### Passer à la correction des sorties d'OCR 

* si fichiers XML

In [56]:
# importer les librairies nécessaires
import os, re, glob, csv
from lxml import etree


# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*')) 


# spécifier les docs d'entrée (échantillon) à partir desquels les sorties corrigées seront générées
directory_in = listdir_nohidden("/content/drive/MyDrive/sample_in/")


# enlever l'extension .xml des fichiers d'entrée 
for file_in in directory_in:
    tree = etree.parse(file_in)
    root = tree.getroot()
    file_in = os.path.basename(file_in)
    file_in = os.path.splitext(file_in)[0]
    # print(file_in) # 5419000_r, test
    

    # créer les nouveaux fichiers .txt sur lesquels les corrections seront appliquées par la suite
    file_out = '{}'.format(file_in)+'_corr.txt'
    # print(file_out) # 5419000_r.txt, test.txt
    directory_out = os.path.join("/content/drive/MyDrive/sample_out/", file_out)
    
    
    # créer les nouveaux fichiers .csv où les erreurs avec leurs corrections seront enregistrées
    corr_out = os.path.join('/content/drive/MyDrive/csv/', file_in+'.csv')
    

    # générer un tableur .csv où les erreurs, les corrections et les fréquences d'erreurs seront stockées
    with open(directory_out, 'w') as f, open(corr_out, 'w') as fout:
      writer = csv.writer(fout)
      writer.writerow(["Erreur"'\t' "Correction"'\t' "Fréquence"'\t']) 
        
    
    # enlever les balises XML afin de transférer le contenu des fichiers .xml dans les fichiers .txt


      for elem in root.iter('*'):
            if elem.text is not None:
                text = elem.text.strip()
                if elem.tail is not None:
                    text = elem.text.strip() + str(elem.tail) # pour récupérer le texte dans la balise imbriquée
                                                               # ex : par le moyen des <hi rend="i">emblèmes</hi>,
                    
                    # pré-traitements
                    text = re.sub('&', 'et', text) # l'esperluette '&' signifie 'et'
                    text = re.sub('« \n', '', text) # pour concaténer les mots séparés par un trait d'union 
                                                    # (sous forme d'un guillemet français ouvert)
                    text = re.sub(" +", " ", text)  # réduire les espaces multiples en un seul espace
                    text = text.lower() # conversion en minuscules
                    text = text.replace("\n", " ") # pour que chaque ligne commence depuis le tout début, 
                                                   # et non pas après un espace
                    
                    # remplacer le guillemet simple par un guillemet français, pour éviter le problème de parsing 
                    text = text.replace("'", "’") 
                    
                    # effacer l'espace avant certains caractères spéciaux
                    text = text.replace(' ,', ',') 
                    text = text.replace(' .','. ')
                    text = text.replace(' :',':')
                    text = text.replace(' ;',';')
                    text = text.replace(' !','!')
                    text = re.sub('\s\?','?', text)
                    text = text.replace(' "','"')
                    text = text.replace('( ','(')
                    text = text.replace(' )',')') 
                    text = text.replace(' –','-')
                    
                    # remplacer les tirets moyens et longs par les tirets courts 
                    text = text.replace('–', '-') 
                    
                    
                    # enlever les signes de ponctuation se trouvant à la fin d'un token
                    # car le signe de ponctuation empêche le correcteur de corriger la séquence 
                    # 'token + signe de ponctuation', même si le token est en effet écrit incorrectement
                    # ex: 'jeuneffe,' (avec virgule) > 'jeuneffe' (incorrect)
                    # au lieu de 'jeuneffe' (sans virgule) > 'jeunesse' (correct)
                    text = re.sub('(?<=\w)[,;:?!.]', '', text)
                    
                    a = jsp.FixFragment(text)        
                    f.write(a + '\n')

    
                    # tokeniser le texte avec le tokeniseur standard (ex: 'l'empire')
                    # car celui de pyspellchecker tokenise mal (ex: 'l', 'empire')
                    token_list = text.split()
                    for t in token_list:
                      m_freq = token_list.count(t)
                      if t not in a:
                        # print(t)
                        fout.write(t+'\t' + jsp.FixFragment(t)+'\t' + str(m_freq) + ' \n')

* si fichiers `.txt`

In [57]:
# importer les librairies nécessaires
import os, re, glob, csv
from lxml import etree


# ignorer les fichiers cachés dans le directoire avec les docs d'entrée (p. ex. le '._5419000_r.xml') 
def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))


# corriger le texte
directory_in = listdir_nohidden("/content/drive/MyDrive/sample_in/")
for file_in in directory_in:
  with open(file_in, 'r') as f:
    text = f.read()
    # print(text)
    a = jsp.FixFragment(text)
    # print(a[:100])

# spécifier les chemins des fichiers corrigés (.txt et .csv)
for file_in in directory_in:
  file_in = os.path.basename(file_in)
  file_in = os.path.splitext(file_in)[0]
  # print(file_in) # Memoires_secrets_01_in.txt

  # créer les nouveaux fichiers .txt sur lesquels les corrections seront appliquées par la suite
  file_out = '{}'.format(file_in)+'_out.txt'
  # print(file_out) # Memoires_secrets_01_in.txt
  directory_out = os.path.join("/content/drive/MyDrive/sample_out/", file_out)
  # print(directory_out) # /content/drive/MyDrive/sample_out/Memoires_secrets_01_in.txt

  
  # créer les nouveaux fichiers .csv où les erreurs avec leurs corrections seront enregistrées
  file_in = os.path.splitext(file_in)[0]
  csv_out = os.path.join('/content/drive/MyDrive/csv/', file_in+'.csv')
  # print(csv_out) # /content/drive/MyDrive/csv/Memoires_secrets_01_in.csv

  with open(directory_out, 'w') as f, open(csv_out, 'w') as fout:
    # writer = csv.writer(fout)
    # writer.writerow(["Erreur"'\t' "Correction"'\t' "Fréquence"'\t']) 

    f.write(a + '\n') # générer le fichier corrigé

    # tokeniser le texte avec le tokeniseur standard (ex: 'l'empire')
    # car celui de pyspellchecker tokenise mal (ex: 'l', 'empire')
    token_list = text.split()
    for t in token_list:
      m_freq = token_list.count(t)
      if t not in a:
        # print(t)
        fout.write(t+'\t' + jsp.FixFragment(t)+'\t' + str(m_freq) + ' \n')